In [1]:
import tensorflow as tf
from PIL import Image
import glob
import numpy as np
import scipy.ndimage.filters
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import scipy.ndimage.filters
import numpy as np
from skimage import feature
import glob
from PIL import Image

from sklearn.externals import joblib


C:\Users\anik\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# image to features
def feature_computation(image):
    features = []
    laplacian = scipy.ndimage.filters.laplace(image)
    lap_mat0 = np.matrix(laplacian[:,:,0])
    lap_mat1 = np.matrix(laplacian[:,:,1])
    lap_mat2 = np.matrix(laplacian[:,:,2])
    
    features.append(lap_mat0.sum())
    features.append(lap_mat1.sum())
    features.append(lap_mat2.sum())
    
    image_gray = image.convert('LA')
    gray_array = np.array(image_gray)[:,:,0]
    edges = feature.canny(gray_array,sigma = 3,low_threshold=5, high_threshold=20)
    max_val = np.amax(edges)

    
    for j in range(0,edges.shape[1]):
        for i in range(0,edges.shape[0]):
            if edges[i][j] > .1*max_val:
                edges[i][j] = 0
                if edges[i+1][j] < .1*max_val:
                    break
        for ii in range(0,edges.shape[0]):
            i = edges.shape[0] - ii -1
            if edges[i][j] > .1*max_val:
                edges[i][j] = 0
                if edges[i-1][j] < .1*max_val:
                    break
                    
    features.append(edges.sum())
    
    return features

    

    
    

In [4]:
image_list_infect = []
for filename in glob.glob('./cell_images/Parasitized/*.png'):
    im=Image.open(filename)
    image_list_infect.append(im.copy())
    im.close()
    

image_list_uninfect = []
for filename in glob.glob('./cell_images/Uninfected/*.png'):
    im=Image.open(filename)
    image_list_uninfect.append(im.copy())
    im.close()

    
data=[]
data_level = []

for i in range(0,len(image_list_infect)):
    data.append(feature_computation(image_list_infect[i]))
    data_level.append(True)
    data.append(feature_computation(image_list_uninfect[i]))
    data_level.append(False)
    
print(len(data_level))
print(len(data))
   

27558
27558


In [5]:
test_length = (int)(.2*len(data))

test_data = data[0:test_length]
test_level = data_level[0:test_length]

train_data = data[test_length:len(data)]
train_level = data_level[test_length:len(data)]


print("train data:" + str(len(train_data)) + " test data: " + str(len(test_data)))



train data:22047 test data: 5511


In [6]:
print(train_level[0:30])

[False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True]


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = [{'n_estimators':[10,15,20,25,30,35,40,45,]}]

forest_class = RandomForestClassifier()
grid_search = GridSearchCV(forest_class,param_grid,cv=5,scoring='accuracy')
grid_search.fit(train_data,train_level)

print(grid_search.best_params_)

model = grid_search.best_estimator_



{'n_estimators': 35}


In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


forest_class = RandomForestClassifier(n_estimators=1)

cross_val_score(forest_class,train_data,train_level,cv=5,scoring='accuracy')




array([1.        , 1.        , 0.99977324, 0.99977319, 1.        ])

In [9]:
from sklearn.metrics import confusion_matrix

test_predict = model.predict(test_data)
print(confusion_matrix(test_level,test_predict))

print(test_level[0:10])
print(test_predict[0:10])


[[2265  490]
 [ 370 2386]]
[True, False, True, False, True, False, True, False, True, False]
[ True False  True False  True False  True False  True False]


In [11]:
from sklearn.externals import joblib

joblib.dump(model,"radom_forest_model.pkl")

['radom_forest_model.pkl']